In [ ]:
# Import required libraries
import pandas as pd
import dash
from dash import html
import dash_core_components as dcc

# Create a dash application
app = dash.Dash(__name__)
app.title = "US Domestic Airline Flights Performance Dashboard"

# Disable callback exception until fully initialized
app.config.suppress_callback_exceptions = True

<ipython-input-9-1842060364>:5: UserWarning:


The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`



In [ ]:
# List of years
year_list = [i for i in range(2005, 2021, 1)]

# Layout for dropdowns
dropdown_layout = html.Div([
    html.Div([
        html.Div([html.H2('Report Type:', style={'margin-right': '2em'})]),
        dcc.Dropdown(
            id='input-type',
            options=[
                {'label': 'Yearly Airline Performance Report', 'value': "OPT1"},
                {'label': 'Yearly Airline Delay Report', 'value': "OPT2"}
            ],
            placeholder='Select a report type',
            style={'text-align': 'center', 'width': '80%', 'padding': '3px', 'font-size': '20px'}
        )
    ], style={'display': 'flex'}),

    html.Div([
        html.Div([html.H2('Choose Year:', style={'margin-right': '2em'})]),
        dcc.Dropdown(
            id='input-year',
            options=[{'label': i, 'value': i} for i in year_list],
            placeholder="Select a year",
            style={'width': '80%', 'padding': '3px', 'font-size': '20px', 'text-align-last': 'center'}
        )
    ], style={'display': 'flex'})
])


In [ ]:
# Output divisions for graphs
output_graphs = html.Div([
    html.Div([], id='plot1'),
    html.Div([
        html.Div([], id='plot2'),
        html.Div([], id='plot3')
    ], style={'display': 'flex'}),
    html.Div([
        html.Div([], id='plot4'),
        html.Div([], id='plot5')
    ], style={'display': 'flex'})
])


In [ ]:
from dash.dependencies import Input, Output, State

@app.callback(
    [Output('plot1', 'children'),
     Output('plot2', 'children'),
     Output('plot3', 'children'),
     Output('plot4', 'children'),
     Output('plot5', 'children')],
    [Input('input-type', 'value'),
     Input('input-year', 'value')],
    [State("plot1", 'children'),
     State("plot2", "children"),
     State("plot3", "children"),
     State("plot4", "children"),
     State("plot5", "children")]
)
def get_graph(chart, year, children1, children2, c3, c4, c5):
    df = airline_data[airline_data['Year'] == year]

    if chart == 'OPT1':
        # Task 2.5 content goes here (see below)
        ...
    else:
        # Task 2.6 content goes here (see below)
        ...


In [ ]:
# Import required libraries
import pandas as pd
import plotly.express as px

# Load data from URL
airline_data = pd.read_csv(
    'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/Data%20Files/airline_data.csv',
    encoding="ISO-8859-1",
    dtype={'Div1Airport': str, 'Div1TailNum': str,
           'Div2Airport': str, 'Div2TailNum': str})

# Define function to compute statistics
def compute_data_choice_1(df):
    # Cancellation Category Count
    bar_data = df.groupby(['Month', 'CancellationCode'])['Flights'].sum().reset_index()

    # Average flight time by reporting airline
    line_data = df.groupby(['Month', 'Reporting_Airline'])['AirTime'].mean().reset_index()

    # Diverted Airport Landings
    div_data = df[df['DivAirportLandings'] != 0.0]

    # Source state count
    map_data = df.groupby(['OriginState'])['Flights'].sum().reset_index()

    # Destination state count
    tree_data = df.groupby(['DestState', 'Reporting_Airline'])['Flights'].sum().reset_index()

    return bar_data, line_data, div_data, map_data, tree_data

# Filter data for a specific year (example: 2010)
df_filtered = airline_data[airline_data['Year'] == 2010]

# Compute data for recession performance report
bar_data, line_data, div_data, map_data, tree_data = compute_data_choice_1(df_filtered)

# Plot 1: Bar Chart - Monthly Flight Cancellation
bar_fig = px.bar(bar_data, x='Month', y='Flights', color='CancellationCode',
                 title='Monthly Flight Cancellation')
bar_fig.show()

# Plot 2: Line Chart - Average Flight Time by Airline
line_fig = px.line(line_data, x='Month', y='AirTime', color='Reporting_Airline',
                   title='Average monthly flight time (minutes) by airline')
line_fig.show()

# Plot 3: Pie Chart - Diverted Airport Landings by Airline
pie_fig = px.pie(div_data, values='Flights', names='Reporting_Airline',
                 title='% of flights by reporting airline (Diverted landings)')
pie_fig.show()

# Plot 4: Choropleth - Flights from Origin State
map_fig = px.choropleth(map_data,
                        locations='OriginState',
                        color='Flights',
                        hover_data=['OriginState', 'Flights'],
                        locationmode='USA-states',
                        color_continuous_scale='GnBu',
                        range_color=[0, map_data['Flights'].max()])
map_fig.update_layout(title_text='Number of flights from origin state', geo_scope='usa')
map_fig.show()

# Plot 5: Treemap - Flights to Destination State by Airline
tree_fig = px.treemap(tree_data,
                      path=['DestState', 'Reporting_Airline'],
                      values='Flights',
                      color='Flights',
                      color_continuous_scale='RdBu',
                      title='Flight count by airline to destination state')
tree_fig.show()


In [ ]:
# Import required libraries
import pandas as pd
import plotly.express as px

# Load data
airline_data = pd.read_csv(
    'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/Data%20Files/airline_data.csv',
    encoding="ISO-8859-1",
    dtype={'Div1Airport': str, 'Div1TailNum': str,
           'Div2Airport': str, 'Div2TailNum': str})

# Define the function to compute delay statistics
def compute_data_choice_2(df):
    # Compute delay averages
    avg_car = df.groupby(['Month', 'Reporting_Airline'])['CarrierDelay'].mean().reset_index()
    avg_weather = df.groupby(['Month', 'Reporting_Airline'])['WeatherDelay'].mean().reset_index()
    avg_NAS = df.groupby(['Month', 'Reporting_Airline'])['NASDelay'].mean().reset_index()
    avg_sec = df.groupby(['Month', 'Reporting_Airline'])['SecurityDelay'].mean().reset_index()
    avg_late = df.groupby(['Month', 'Reporting_Airline'])['LateAircraftDelay'].mean().reset_index()

    return avg_car, avg_weather, avg_NAS, avg_sec, avg_late

# Filter data for a specific year (example: 2015)
df_filtered = airline_data[airline_data['Year'] == 2015]

# Compute the delay data
avg_car, avg_weather, avg_NAS, avg_sec, avg_late = compute_data_choice_2(df_filtered)

# Plot 1: Carrier Delay
carrier_fig = px.line(avg_car, x='Month', y='CarrierDelay', color='Reporting_Airline',
                      title='Average carrier delay time (minutes) by airline')
carrier_fig.show()

# Plot 2: Weather Delay
weather_fig = px.line(avg_weather, x='Month', y='WeatherDelay', color='Reporting_Airline',
                      title='Average weather delay time (minutes) by airline')
weather_fig.show()

# Plot 3: NAS Delay
nas_fig = px.line(avg_NAS, x='Month', y='NASDelay', color='Reporting_Airline',
                  title='Average NAS delay time (minutes) by airline')
nas_fig.show()

# Plot 4: Security Delay
sec_fig = px.line(avg_sec, x='Month', y='SecurityDelay', color='Reporting_Airline',
                  title='Average security delay time (minutes) by airline')
sec_fig.show()

# Plot 5: Late Aircraft Delay
late_fig = px.line(avg_late, x='Month', y='LateAircraftDelay', color='Reporting_Airline',
                   title='Average late aircraft delay time (minutes) by airline')
late_fig.show()


The error `ModuleNotFoundError: No module named 'dash'` means that the Python environment you are using does not have the `dash` library installed.

The cell above installs `dash` and `pandas`, which are necessary for the subsequent code to run.

# Task
Modify the code in the code interpreter to display all generated plots side-by-side in a single figure.

## Mdknlhe kvzp

### Subtask:
Import `make_subplots` from `plotly.subplots`, create a single figure with multiple subplots, add each generated plot as a trace to the appropriate subplot, update the layout, and display the combined figure.


**Reasoning**:
The subtask requires combining multiple plots into a single figure using subplots. This involves importing `make_subplots`, creating the subplot figure, adding traces from existing plots to the subplots, and updating the layout.



In [ ]:
from plotly.subplots import make_subplots

# Create a figure with multiple subplots
fig = make_subplots(rows=3, cols=2,
                    specs=[[{"colspan": 2}, None],
                           [{}, {}],
                           [{}, {}]],
                    subplot_titles=('Monthly Flight Cancellation', 'Average monthly flight time (minutes) by airline',
                                    '% of flights by reporting airline (Diverted landings)',
                                    'Number of flights from origin state', 'Flight count by airline to destination state'))

# Add traces from individual plots to the subplots
for trace in bar_fig['data']:
    fig.add_trace(trace, row=1, col=1)

for trace in line_fig['data']:
    fig.add_trace(trace, row=2, col=1)

for trace in pie_fig['data']:
    fig.add_trace(trace, row=2, col=2)

for trace in map_fig['data']:
    fig.add_trace(trace, row=3, col=1)

for trace in tree_fig['data']:
    fig.add_trace(trace, row=3, col=2)

# Update layout
fig.update_layout(title_text="Yearly Airline Performance Report - 2010", height=1000)

# Display the combined figure
fig.show()

ValueError: Trace type 'pie' is not compatible with subplot type 'xy'
at grid position (2, 2)

See the docstring for the specs argument to plotly.subplots.make_subplots
for more information on subplot types

**Reasoning**:
The traceback indicates that a pie chart cannot be added to a subplot of type 'xy'. The `specs` argument in `make_subplots` needs to be updated to specify the type of subplot for the pie chart.



In [ ]:
from plotly.subplots import make_subplots

# Create a figure with multiple subplots
fig = make_subplots(rows=3, cols=2,
                    specs=[[{"colspan": 2}, None],
                           [{}, {}],
                           [{}, {}]],
                    subplot_titles=('Monthly Flight Cancellation', 'Average monthly flight time (minutes) by airline',
                                    '% of flights by reporting airline (Diverted landings)',
                                    'Number of flights from origin state', 'Flight count by airline to destination state'))

# Update specs to include 'domain' type for the pie chart subplot
fig = make_subplots(rows=3, cols=2,
                    specs=[[{"colspan": 2}, None],
                           [{}, {'type': 'domain'}],
                           [{}, {}]],
                    subplot_titles=('Monthly Flight Cancellation', 'Average monthly flight time (minutes) by airline',
                                    '% of flights by reporting airline (Diverted landings)',
                                    'Number of flights from origin state', 'Flight count by airline to destination state'))

# Add traces from individual plots to the subplots
for trace in bar_fig['data']:
    fig.add_trace(trace, row=1, col=1)

for trace in line_fig['data']:
    fig.add_trace(trace, row=2, col=1)

for trace in pie_fig['data']:
    fig.add_trace(trace, row=2, col=2)

for trace in map_fig['data']:
    fig.add_trace(trace, row=3, col=1)

for trace in tree_fig['data']:
    fig.add_trace(trace, row=3, col=2)


# Update layout
fig.update_layout(title_text="Yearly Airline Performance Report - 2010", height=1000)

# Display the combined figure
fig.show()

ValueError: Trace type 'choropleth' is not compatible with subplot type 'xy'
at grid position (3, 1)

See the docstring for the specs argument to plotly.subplots.make_subplots
for more information on subplot types

**Reasoning**:
The traceback indicates that a choropleth map cannot be added to a subplot of type 'xy'. The `specs` argument in `make_subplots` needs to be updated to specify the type of subplot for the choropleth map.



In [ ]:
from plotly.subplots import make_subplots

# Create a figure with multiple subplots
fig = make_subplots(rows=3, cols=2,
                    specs=[[{"colspan": 2}, None],
                           [{}, {'type': 'domain'}],
                           [{'type': 'domain'}, {}]],
                    subplot_titles=('Monthly Flight Cancellation', 'Average monthly flight time (minutes) by airline',
                                    '% of flights by reporting airline (Diverted landings)',
                                    'Number of flights from origin state', 'Flight count by airline to destination state'))


# Add traces from individual plots to the subplots
for trace in bar_fig['data']:
    fig.add_trace(trace, row=1, col=1)

for trace in line_fig['data']:
    fig.add_trace(trace, row=2, col=1)

for trace in pie_fig['data']:
    fig.add_trace(trace, row=2, col=2)

for trace in map_fig['data']:
    fig.add_trace(trace, row=3, col=1)

for trace in tree_fig['data']:
    fig.add_trace(trace, row=3, col=2)


# Update layout
fig.update_layout(title_text="Yearly Airline Performance Report - 2010", height=1000)

# Display the combined figure
fig.show()

ValueError: Trace type 'choropleth' is not compatible with subplot type 'domain'
at grid position (3, 1)

See the docstring for the specs argument to plotly.subplots.make_subplots
for more information on subplot types

**Reasoning**:
The traceback indicates that a choropleth map is not compatible with a 'domain' subplot type. The `specs` argument needs to be updated to use the 'choropleth' subplot type for the map.



In [ ]:
from plotly.subplots import make_subplots

# Create a figure with multiple subplots
fig = make_subplots(rows=3, cols=2,
                    specs=[[{"colspan": 2}, None],
                           [{}, {'type': 'domain'}],
                           [{'type': 'choropleth'}, {}]],
                    subplot_titles=('Monthly Flight Cancellation', 'Average monthly flight time (minutes) by airline',
                                    '% of flights by reporting airline (Diverted landings)',
                                    'Number of flights from origin state', 'Flight count by airline to destination state'))


# Add traces from individual plots to the subplots
for trace in bar_fig['data']:
    fig.add_trace(trace, row=1, col=1)

for trace in line_fig['data']:
    fig.add_trace(trace, row=2, col=1)

for trace in pie_fig['data']:
    fig.add_trace(trace, row=2, col=2)

for trace in map_fig['data']:
    fig.add_trace(trace, row=3, col=1)

for trace in tree_fig['data']:
    fig.add_trace(trace, row=3, col=2)


# Update layout
fig.update_layout(title_text="Yearly Airline Performance Report - 2010", height=1000)

# Display the combined figure
fig.show()

ValueError: Trace type 'treemap' is not compatible with subplot type 'xy'
at grid position (3, 2)

See the docstring for the specs argument to plotly.subplots.make_subplots
for more information on subplot types

**Reasoning**:
The traceback indicates that a treemap is not compatible with a 'xy' subplot type. The `specs` argument in `make_subplots` needs to be updated to specify the type of subplot for the treemap.



In [ ]:
from plotly.subplots import make_subplots

# Create a figure with multiple subplots
fig = make_subplots(rows=3, cols=2,
                    specs=[[{"colspan": 2}, None],
                           [{}, {'type': 'domain'}],
                           [{'type': 'choropleth'}, {'type': 'domain'}]],
                    subplot_titles=('Monthly Flight Cancellation', 'Average monthly flight time (minutes) by airline',
                                    '% of flights by reporting airline (Diverted landings)',
                                    'Number of flights from origin state', 'Flight count by airline to destination state'))


# Add traces from individual plots to the subplots
for trace in bar_fig['data']:
    fig.add_trace(trace, row=1, col=1)

for trace in line_fig['data']:
    fig.add_trace(trace, row=2, col=1)

for trace in pie_fig['data']:
    fig.add_trace(trace, row=2, col=2)

for trace in map_fig['data']:
    fig.add_trace(trace, row=3, col=1)

for trace in tree_fig['data']:
    fig.add_trace(trace, row=3, col=2)


# Update layout
fig.update_layout(title_text="Yearly Airline Performance Report - 2010", height=1000)

# Display the combined figure
fig.show()

## Modify `hxlnvub1kwko`

### Subtask:
Import `make_subplots` from `plotly.subplots`, create a single figure with multiple subplots, add each generated line plot as a trace to the appropriate subplot, update the layout, and display the combined figure.


**Reasoning**:
Import `make_subplots` from `plotly.subplots`, create a single figure with multiple subplots, add each generated line plot as a trace to the appropriate subplot, update the layout, and display the combined figure.



In [ ]:
from plotly.subplots import make_subplots

# Create a figure with multiple subplots
fig = make_subplots(rows=3, cols=2,
                    subplot_titles=('Average carrier delay time by airline',
                                    'Average weather delay time by airline',
                                    'Average NAS delay time by airline',
                                    'Average security delay time by airline',
                                    'Average late aircraft delay time by airline'))

# Add traces from individual plots to the subplots
for trace in carrier_fig['data']:
    fig.add_trace(trace, row=1, col=1)

for trace in weather_fig['data']:
    fig.add_trace(trace, row=1, col=2)

for trace in nas_fig['data']:
    fig.add_trace(trace, row=2, col=1)

for trace in sec_fig['data']:
    fig.add_trace(trace, row=2, col=2)

for trace in late_fig['data']:
    fig.add_trace(trace, row=3, col=1)

# Update layout
fig.update_layout(title_text="Yearly Airline Delay Report - 2015", height=900)

# Display the combined figure
fig.show()

## Summary:

### Data Analysis Key Findings

*   The analysis successfully combined five different plots (bar, line, pie, choropleth map, and treemap) into a single figure using `plotly.subplots`.
*   Specific subplot types (`domain` for pie and treemap, `choropleth` for the map) were required in the `make_subplots` `specs` argument for successful rendering of non-'xy' plot types.
*   Five line plots showing different types of flight delays were also successfully combined into a single figure using `plotly.subplots` with a 3x2 grid layout.

### Insights or Next Steps

*   Using `plotly.subplots` is an effective way to present multiple related visualizations in a single view for easier comparison and reporting.
*   When combining different plot types, it is crucial to specify the correct `specs` for `make_subplots` to ensure compatibility with the trace types being added.
